<a href="https://colab.research.google.com/github/JacopoMangiavacchi/FastLabeling/blob/master/mnist_odd_even_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import FastAI and MNIST dataset

In [1]:
!pip install fastai -U

     |████████████████████████████████| 358kB 2.8MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [2]:
from fastai.vision.all import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path

# Utility Function to create subset copies of the MNIST dataset with different labels

In [4]:
from shutil import copy2, rmtree

def copy_files(path, source_folder, destination_folder, sub_folders_list = [], random_select = False, max_files_to_copy = 0):
  def copy_folder_path(source_path, destination_path):
    source_files = source_path.ls()
    list_files = list(range(len(source_files)))
    if random_select:
      random.shuffle(list_files)
    count = 0
    for i in list_files:
      copy2(source_files[i], destination_path)
      count += 1
      if max_files_to_copy > 0 and count >= max_files_to_copy:
        break
  
  (path/destination_folder).mkdir(exist_ok=True)
  if len(sub_folders_list) > 0:
    for i in sub_folders_list:
      folder = source_folder + '/' + str(i)
      copy_folder_path(path/folder, path/destination_folder)
  else:
    copy_folder_path(path/source_folder, path/destination_folder)


# Prepare DataLoaders from transformed MNSIT with ODD and EVEN Labels instead of DIGITS

In [5]:
rmtree(path/'oe_training', ignore_errors=true)
rmtree(path/'oe_testing', ignore_errors=true)

(path/'oe_training').mkdir(exist_ok=True)
(path/'oe_testing').mkdir(exist_ok=True)
path.ls()

(#4) [Path('testing'),Path('oe_testing'),Path('oe_training'),Path('training')]

In [6]:
copy_files(path, 'training', 'oe_training/odd', [0, 2, 4, 6, 8])
copy_files(path, 'training', 'oe_training/even', [1, 3, 5, 7, 9])
copy_files(path, 'testing', 'oe_testing/odd', [0, 2, 4, 6, 8])
copy_files(path, 'testing', 'oe_testing/even', [1, 3, 5, 7, 9])


In [7]:
(path/'oe_training/odd').ls(), (path/'oe_training/even').ls(), (path/'oe_testing/odd').ls(), (path/'oe_testing/even').ls()

((#29492) [Path('oe_training/odd/36982.png'),Path('oe_training/odd/15882.png'),Path('oe_training/odd/13101.png'),Path('oe_training/odd/16752.png'),Path('oe_training/odd/56894.png'),Path('oe_training/odd/4036.png'),Path('oe_training/odd/21084.png'),Path('oe_training/odd/51453.png'),Path('oe_training/odd/8261.png'),Path('oe_training/odd/3046.png')...],
 (#30508) [Path('oe_training/even/28988.png'),Path('oe_training/even/22542.png'),Path('oe_training/even/47998.png'),Path('oe_training/even/26376.png'),Path('oe_training/even/45534.png'),Path('oe_training/even/38894.png'),Path('oe_training/even/27277.png'),Path('oe_training/even/10555.png'),Path('oe_training/even/31418.png'),Path('oe_training/even/18059.png')...],
 (#4926) [Path('oe_testing/odd/3046.png'),Path('oe_testing/odd/4439.png'),Path('oe_testing/odd/9271.png'),Path('oe_testing/odd/9086.png'),Path('oe_testing/odd/644.png'),Path('oe_testing/odd/6211.png'),Path('oe_testing/odd/7305.png'),Path('oe_testing/odd/4354.png'),Path('oe_testing

In [8]:
even_odd_dls = ImageDataLoaders.from_folder(path, train='oe_training', valid='oe_testing')

even_odd_dls.train_ds, even_odd_dls.valid_ds

((#60000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1))...],
 (#10000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size

# Train Full MNIST Dataset on ResNet architecture with modified labels ODD and EVEN

In [9]:
learn = cnn_learner(even_odd_dls, resnet34, metrics=accuracy)
learn.fine_tune(3)


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


epoch,train_loss,valid_loss,accuracy,time
0,0.277041,0.216326,0.913200,01:57


epoch,train_loss,valid_loss,accuracy,time
0,0.049738,0.031313,0.988900,02:09
1,0.022741,0.013498,0.995300,02:10
2,0.010502,0.014450,0.994900,02:11


# Prepare DataLoaders for a small subset of the MNIST Dataset with real digit labels for only 60 samples per class (3 order of magnitute smaller)

In [10]:
size_traning = 60
size_testing = 10
size_training_name = str(size_traning) + '_training'
size_testing_name = str(size_testing) + '_testing'

rmtree(path/size_training_name, ignore_errors=true)
rmtree(path/size_testing_name, ignore_errors=true)

(path/size_training_name).mkdir(exist_ok=True)
(path/size_testing_name).mkdir(exist_ok=True)
path.ls()

(#6) [Path('testing'),Path('10_testing'),Path('oe_testing'),Path('60_training'),Path('oe_training'),Path('training')]

In [11]:
for i in range(10):
  copy_files(path, 'training/' + str(i), size_training_name + '/' + str(i), random_select=True, max_files_to_copy=size_traning)
  copy_files(path, 'testing/' + str(i), size_testing_name + '/' + str(i), random_select=True, max_files_to_copy=size_testing)

In [12]:
subset_dls = ImageDataLoaders.from_folder(path, train=size_training_name, valid=size_testing_name)

subset_dls.train_ds, subset_dls.valid_ds

((#600) [(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2))...],
 (#100) [(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x

# Transfer Learning reusing the ResNet model previosly trained on EVEN / ODD labels and change last Linear learner to fine-tune on the 10 digit labels and train on the new small subset dataset

In [13]:
learn.model[1][8]

Linear(in_features=512, out_features=2, bias=False)

In [14]:
learn.model[1][8] = torch.nn.modules.linear.Linear(in_features=512, out_features=10, bias=False)

In [15]:
learn2 = Learner(subset_dls, learn.model, metrics=accuracy)
learn2.fine_tune(20)


epoch,train_loss,valid_loss,accuracy,time
0,1.610765,1.880049,0.370000,00:02


epoch,train_loss,valid_loss,accuracy,time
0,0.394862,0.651945,0.830000,00:01
1,0.295677,0.263021,0.930000,00:01
2,0.241495,0.183822,0.970000,00:01
3,0.196587,0.368697,0.930000,00:01
4,0.209737,0.249532,0.920000,00:01
5,0.212575,0.319085,0.920000,00:01
6,0.221468,1.296736,0.740000,00:01
7,0.227926,0.186414,0.950000,00:01
8,0.217985,0.402959,0.900000,00:01
9,0.196560,0.207718,0.950000,00:01


In [16]:
test_path = (path/'testing'/'5').ls()[10]
test_image = Image.open(test_path)
test_image

In [17]:
learn2.predict(test_path)

('5',
 tensor(5),
 tensor([1.2722e-04, 2.7529e-04, 1.6198e-04, 2.6956e-04, 1.3667e-04, 9.9791e-01,
         4.7790e-04, 9.5492e-05, 1.2841e-04, 4.1452e-04]))

# Classic Transfer Learning training from original ResNet on the new small subset dataset

In [18]:
learn3 = cnn_learner(subset_dls, resnet34, metrics=accuracy)
learn3.fine_tune(20)

epoch,train_loss,valid_loss,accuracy,time
0,3.502641,2.090457,0.290000,00:01


epoch,train_loss,valid_loss,accuracy,time
0,2.218468,1.978958,0.380000,00:01
1,1.966882,1.779201,0.490000,00:01
2,1.703303,1.500109,0.570000,00:01
3,1.465718,1.219936,0.650000,00:01
4,1.218771,1.098952,0.700000,00:01
5,1.008214,1.058523,0.710000,00:01
6,0.841556,0.933104,0.730000,00:01
7,0.692130,0.857036,0.750000,00:01
8,0.581191,0.802711,0.770000,00:01
9,0.486998,0.833801,0.770000,00:01


# Prepare DataLoaders for full MNIST dataset with original labels



In [19]:
full_dls = ImageDataLoaders.from_folder(path, train='training', valid='testing')

full_dls.train_ds, full_dls.valid_ds


((#60000) [(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2))...],
 (#10000) [(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size=28x28, TensorCategory(2)),(PILImage mode=RGB size

# Benchmarking the two models with the full MNIST validation dataset

In [21]:
learn2.validate(dl=full_dls.valid)

(#2) [0.09005868434906006,0.975600004196167]

In [23]:
learn3.validate(dl=full_dls.valid)

(#2) [11.524850845336914,0.08940000087022781]